# Traditional Methods trained on every file, tested on one at a time (untuned)
MinMaxScaler is applied to these tests. All models are untuned. The model will be trained on file A, tested on files B,C,D.., then on B, tested on A,C,D.. etc

In [1]:
import sys
sys.path.append("../../")
from time import process_time
from os import listdir, chdir, environ, path

def warn(*args, **kwargs):
  pass
import warnings
warnings.warn = warn

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
from modules.NetworkTraffic import NT2
from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from autosklearn.experimental.askl2 import AutoSklearn2Classifier

FilesToTest = list()
chdir("../../data/other")
for file in listdir():
  if file.endswith(".csv"):
    if file.startswith('b'):
      FilesToTest.append(file)

TestSize = [0.4]
ModelsToTest = [AutoSklearn2Classifier(time_left_for_this_task=60, n_jobs=-1, memory_limit=4096)]
OutputResults = dict()

In [2]:
theseFiles = dict()
for file in FilesToTest:
  theseFiles[file] = NT2(file, transform=False, drop=True)

In [3]:
import pandas as pd
from copy import deepcopy
df_leader = None

def testMe():
  global df_leader
  df_leader = pd.DataFrame()
  OutputResults.clear()
  #ModelResults.clear()

  for index, file in enumerate(FilesToTest):
    print(file, end=' ')
    OutputResults[file] = dict()
    currentFileData = theseFiles[file]
    restOfFiles = deepcopy(FilesToTest)
    restOfFiles.pop(index)

    for model in ModelsToTest:
      model.fit(currentFileData.data, currentFileData.target)
      print('[', end=' ')
      for file2 in restOfFiles:
        print('.', end=' ')
        OutputResults[file][file2] = dict()
        testFileData = theseFiles[file2]
        y_pred = model.predict(testFileData.data)
        score = metrics.accuracy_score(testFileData.target, y_pred)
        OutputResults[file][file2]["Accuracy"] = score
        try:
          OutputResults[file][file2]["Final Ensemble"] = str(model.show_models())
        except:
          pass
        try:
          if df_leader.empty: df_leader = model.leaderboard()
          else: df_leader = pd.concat([df_leader, model.leaderboard()], ignore_index=False)
        except: pass
        try:
          OutputResults[file][file2]["Sprint"] = str(model.sprint_statistics())
        except KeyError:
          pass
    print(']')

In [4]:
import json
from os import path

def writeMe():
  with open("EveryFileTransfer_Untuned_AllTestResults.json", "a") as f:
    f.write(json.dumps(OutputResults, indent=2))

  df_leader.to_csv("Leaderboard.csv", mode='a')

  with open("EveryFileTransfer_Untuned_ModelResults.csv", "a") as f3:
    if path.exists("EveryFileTransfer_Untuned_ModelResults.csv"): f3.write("Trained On,Tested On,Model,Accuracy,Runtime\n")
    for file in OutputResults:
      for file2 in OutputResults[file]:
        m = str(ModelsToTest[0]).split('(')[0]
        f3.write(f"{file},{file2},{m},{OutputResults[file][file2]['Accuracy']}\n")

In [5]:
REPEATS = 2
for _ in range(0, REPEATS):
  testMe()
  writeMe()

b1000d60.csv [ . ]
b1000d35.csv [ . ]
b1000d60.csv [ . ]
b1000d35.csv [ . ]
